# This script aims to prototype all parametrizations which are necessary for mechanical models

In [13]:
from lxml import etree as et
import os
import pandas as pd
caaml_ns = "{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}"

def mm2m(df):
    return(df/1000)

In [2]:
smp_deriv_file_path = '.pnt_data\\190110_BUN\\SMP\\FILE0245_smp_ful.pkl'  # Replace with a template file_path
layered_file_path = '.pnt_data\\190110_BUN\\SMP\\FILE0245_smp_lay.pkl'  # Replace with a template file_path

# read the SMP micromechanical properties from pickle

In [8]:
df_pkl = pd.read_pickle(smp_deriv_file_path)
df_pkl_layered = pd.read_pickle(layered_file_path)

In [9]:
from Emod_parametrizations import e_gerling_2017_AC
df_pkl_layered['G2017_E_AC'] = df_pkl_layered['CR2020_density'].apply(e_gerling_2017_AC) 

### Parametrizations from Johnson Schneebeli 1999 (Characterizing the microstructural and micromechanical properties of snow)

In [10]:
df_pkl_layered["JS1999_sigma_n"] = df_pkl_layered.L2012_f0 / (df_pkl_layered.L2012_L * df_pkl_layered.L2012_L)
df_pkl_layered["JS1999_sigma_macro"] = df_pkl_layered.JS1999_sigma_n * (df_pkl_layered.L2012_delta / df_pkl_layered.L2012_L)

### --> compute load of for each layer

In [27]:
layer_weight = df_pkl_layered.CR2020_density * mm2m(df_pkl_layered.thickness) # kg/m^2
layer_weight = layer_weight.cumsum().shift(1, fill_value=0) # adding up the weight and just taking the weights above
layer_weight = layer_weight * 9.81 # kg/m^2 * m/s^2 = N/m^2 = Pa
df_pkl_layered["load_above"] = layer_weight
df_pkl_layered.load_above.attrs['unit'] = "kg/m^2"

In [ ]:
df_pkl_layered.load_above.attrs['unit'] = "kg/m^2"

### --> use weac to compute (skier) shear stress for each layer

In [3]:
def read_from_caaml(filepath, prof_type):
    """small CAAML reader This routine reads the :prof_type data from a CAAML XML file:
        Most likely the values are parameterized from a shot noise model for the
        SMP forces. However it also can read manual profiles:

    :param filepath: Filename of the *.caaml file.
    :param prof_type: either "specSurfArea" or density "density" 
    """
   
    tree = et.parse(file_path)
    xroot = tree.getroot()
    data = []
    child_snowProfileResultsOf = xroot.find(caaml_ns + 'snowProfileResultsOf')
    child_SnowProfileMeasurements = child_snowProfileResultsOf.find(caaml_ns + 'SnowProfileMeasurements')
    child_Profile = child_SnowProfileMeasurements.find(caaml_ns + prof_type + 'Profile')
    child_Layer = child_Profile.find(caaml_ns + 'Layer')  
    for layer in child_Profile:
        try:
            data.append({
            "depthTop": float(layer.find(caaml_ns + 'depthTop').text),
            "thickness": float(layer.find(caaml_ns + 'thickness').text),
            prof_type: float(layer.find(caaml_ns + prof_type).text)
        })
        except:
            pass
    df = pd.DataFrame(data)
    return(df)
    
df_caaml = read_from_caaml(file_path, "density").merge(read_from_caaml(file_path, "specSurfArea"))
df_caaml

,depthTop,thickness,density,specSurfArea
0,0.0,15.1,122.65,52.93
1,15.1,0.2,178.25,39.21
2,15.3,0.1,163.13,47.18
3,15.4,2.4,167.79,39.91
4,17.8,0.3,151.16,44.91
5,18.1,0.8,143.80,40.90
6,18.9,0.1,132.91,43.93
7,19.0,4.1,156.84,39.25
8,23.1,0.1,138.50,43.60
9,23.2,2.3,154.81,39.03
